In [1]:
import numpy as np
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
import pickle
import tqdm
import config
conf, _ = config.get_config()

c:\users\mmnet\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Namespace(MAX_ITERATION=100000.0, batch_size=100, gpu=1, lr=0.0001, tar_model='model_1', training=True)


In [2]:
'''Setting visable GPU'''
gpus = [conf.gpu] # Here I set CUDA to only see one GPU
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(i) for i in gpus])

In [3]:
model_dir = './'+conf.tar_model+'/'
data_dir = './dataset/'
logs_dir = model_dir + 'logs/'

In [4]:
if conf.tar_model == 'model_1':
    import model_1 as model
    print('Load', conf.tar_model)
else:
    sys.exit("Sorry, Wrong Model!")

Load model_1


# Read dataset

In [5]:
# from sparse matrix
# temp = mmread('./sparse_behavior_data.csv')
# den_data = temp.toarray().astype(np.uint16)

# from dense array
mnist = tf.contrib.learn.datasets.load_dataset('mnist')

raw_data = mnist.train.images # Returns np.array

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
# check shapes of raw_data
print(type(raw_data))
print(raw_data.shape)

<class 'numpy.ndarray'>
(55000, 784)


In [7]:
# preprocessing for the data
processed_data = raw_data

In [8]:
# check shapes of processed_data
print(type(processed_data))
print(processed_data.shape)
#den_data=den_data.astype(np.uint16)

<class 'numpy.ndarray'>
(55000, 784)


In [9]:
# setting a batch generator for getting a batch of dataset
def batch_generator(data, idx, batch_size = 128):
    # while (1):
    while(True):
        np.random.shuffle(idx)
        for i in range(int(len(idx)/batch_size)):
            yield data[idx[i*batch_size:((i+1)*batch_size)]]
            # you might have a training label, and the output will be 
            # yield traing_data[idx[i*batch_size:((i+1)*batch_size)]], yield traing_label[idx[i*batch_size:((i+1)*batch_size)]]
            
# split training and validation set
def get_generators(training_data, tv_ratio = 0.8, shuffle = True, batch_size = 128):
    
    print('Size of all training data', training_data.shape)
    idx = np.arange(0, training_data.shape[0], 1)
    if shuffle:
        print('*Shuffle before training and validation split')
        np.random.shuffle(idx)
    training_idx = idx[:(int(tv_ratio*len(idx)))]
    validation_idx =idx[int(tv_ratio*len(idx)):]
    if (len(training_idx) + len(validation_idx) - len(idx)) > 0:
        print('Some data are duplicated in the training and validation split')
    elif (len(training_idx) + len(validation_idx) - len(idx)) < 0:
        print('Some data are missing in the training and validation split')

    training_gen = batch_generator(training_data, training_idx, batch_size)
    validation_gen = batch_generator(training_data, validation_idx, batch_size)
    return training_gen, len(training_idx), validation_gen, len(validation_idx)

In [10]:
# defining the batch generators
if conf.training:
    get_training_batch, n_training_samples, get_validation_batch, n_validation_samples = get_generators(processed_data, tv_ratio = 0.8, shuffle = True, batch_size = conf.batch_size) # (128, 30, 450)
else:
    testing_idx = np.arange(0, processed_data.shape[0], 1)
    get_testing_batch = batch_generator(processed_data, testing_idx, conf.batch_size)

Size of all training data (55000, 784)
*Shuffle before training and validation split


In [11]:
# b = next(get_training_batch)
# print(b.shape)
# print(n_training_samples)

# b = next(get_validation_batch)
# print(b.shape)
# print(n_validation_samples)

# Training

In [ ]:
# clear tensorflow graph
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default() as g:
    
    ###############
    # Model Setup #
    ###############
    
    # model input
    model_input = tf.placeholder(tf.float32, shape=[conf.batch_size, 784], name="model_input")
    model_GT = tf.placeholder(tf.float32, shape=(conf.batch_size, 784), name='model_GT')
    keep_probability = tf.placeholder(tf.float32, name="keep_probabilty")
    train_phase = tf.placeholder(tf.bool, name='phase_train')

    # build model
    code, model_out = model.inference(model_input, keep_probability, train_phase)

    # check # parameter of the model
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        #   print(shape)
        #   print(len(shape))
        variable_parameters = 1
        for dim in shape:
        #   print(dim)
            variable_parameters *= dim.value
        #   print(variable_parameters)
        total_parameters += variable_parameters
    print('total_parameters', total_parameters)
    
    # model loss
    loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions=model_out, labels=model_GT))
    loss_summary = tf.summary.scalar("Tot_loss", loss)

    def train(loss_val, var_list):
        optimizer = tf.train.AdamOptimizer(conf.lr)
        grads = optimizer.compute_gradients(loss_val, var_list=var_list)
        return optimizer.apply_gradients(grads)

    trainable_var = tf.trainable_variables()
    
    train_op = train(loss, trainable_var)
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    # setup
    print("Setting up summary op...")
    summary_op = tf.summary.merge_all()

    print("Setting up Saver...")
    saver = tf.train.Saver()

    sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False), graph=g)
    
    if (conf.training == False):
        ckpt = tf.train.get_checkpoint_state(logs_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            print('Loading sucessfully')
        else:
            print('No checkpoint file found')
            raise
    else:
        init = tf.global_variables_initializer()
        sess.run(init)

    # create two summary writers to show training loss and validation loss in the same graph
    # need to create two folders 'train' and 'validation' inside FLAGS.logs_dir
    train_writer = tf.summary.FileWriter(logs_dir+'/train', sess.graph)
    validation_writer = tf.summary.FileWriter(logs_dir+'/validation', sess.graph)
    
    #########
    # Start #
    #########
    if conf.training == True:
        print("-------- Start Training --------")
        max_validloss = 99999
        for itr in range(int(conf.MAX_ITERATION)):
            # prepare training input
            batch_xs = next(get_training_batch)

            sess.run([train_op,extra_update_ops], feed_dict={model_input: batch_xs,
                                                             model_GT: batch_xs,
                                                             keep_probability: 0.85,
                                                             train_phase:conf.training})

            if itr % 2000 == 0:
                train_loss, summary_str = sess.run([loss, loss_summary], feed_dict={model_input: batch_xs,
                                                                                     model_GT: batch_xs,
                                                                                     keep_probability: 1,
                                                                                     train_phase:False})
                print("[T] Step: %d, loss:%g" % (itr, np.mean(train_loss)))
                train_writer.add_summary(summary_str, itr)
                
            # validation
            if itr % 10000 == 0:
                # prepare inputs
                valid_losses = []
                for i in tqdm.trange(int(n_validation_samples/conf.batch_size)):
                    batch_xs_valid = next(get_validation_batch)
                    
                    valid_loss, summary_sva=sess.run([loss, loss_summary], feed_dict={model_input: batch_xs_valid,
                                                                                     model_GT: batch_xs_valid,
                                                                                     keep_probability: 1,
                                                                                     train_phase:False})
                    valid_losses.append(valid_loss)
                    
                # save validation log
                validation_writer.add_summary(summary_sva, itr)
                # save the ckpt if reachings better loss
                calc_v_loss = np.mean(valid_losses)

                if calc_v_loss < max_validloss:
                    saver.save(sess, logs_dir + "model.ckpt", itr)
                    print("[V*] Step: %d, loss:%g" % (itr, calc_v_loss))
                    max_validloss = calc_v_loss
                else:
                    print("[V] Step: %d, loss:%g" % (itr, calc_v_loss))
    else:
        print("Start Testing")

        test_losses = []
        codes=[]
        for i in tqdm.trange(int(processed_data.shape[0]/conf.batch_size)):
            batch_xs_test = next(get_testing_batch)

            coder, test_loss=sess.run([code, loss], feed_dict={model_input: batch_xs_test,
                                                                             model_GT: batch_xs_test,
                                                                             keep_probability: 1,
                                                                             train_phase:False})
            test_losses.append(test_loss)
            codes.append(coder)


        # save the ckpt if reachings better loss
        calc_test_loss = np.mean(test_losses)
        print("[Test] Avg. loss:%g" % (calc_test_loss))
        a = np.array(codes); print(a.shape)
        b = np.swapaxes(a, 0, 1); print(b.shape)
        c = b.reshape(-1, b.shape[2]);print(c.shape)
        import pandas
        encode = pandas.DataFrame(c)
        encode.to_csv('encode.csv',index = False)
        
    sess.close()


total_parameters 942656
Setting up summary op...
Setting up Saver...
-------- Start Training --------
[T] Step: 0, loss:0.113827


100%|███████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 746.28it/s]


[V*] Step: 0, loss:0.113011
[T] Step: 2000, loss:0.0359481
[T] Step: 4000, loss:0.028422
[T] Step: 6000, loss:0.0243834
[T] Step: 8000, loss:0.0232882
[T] Step: 10000, loss:0.0196861


100%|███████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 729.30it/s]


[V*] Step: 10000, loss:0.0209793
[T] Step: 12000, loss:0.0186628
[T] Step: 14000, loss:0.0187184
[T] Step: 16000, loss:0.0177832
[T] Step: 18000, loss:0.0176104
[T] Step: 20000, loss:0.0170484


...

[V*] Step: 50000, loss:0.0141388
[T] Step: 52000, loss:0.0129673
[T] Step: 54000, loss:0.0123059
[T] Step: 56000, loss:0.0140105
[T] Step: 58000, loss:0.0132247
[T] Step: 60000, loss:0.0119239


100%|███████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 726.42it/s]


[V*] Step: 60000, loss:0.0136684
[T] Step: 62000, loss:0.0124587
[T] Step: 64000, loss:0.0124789
[T] Step: 66000, loss:0.0118648
[T] Step: 68000, loss:0.0131211
[T] Step: 70000, loss:0.0115306


100%|███████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 725.93it/s]


[V*] Step: 70000, loss:0.013314
[T] Step: 72000, loss:0.0132774
[T] Step: 74000, loss:0.0123373
[T] Step: 76000, loss:0.0127052
[T] Step: 78000, loss:0.0125724
